<a href="https://colab.research.google.com/github/amitvashista/nifty_50/blob/stage1/Deep_Q_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv("/content/NiftyOptionCE_2025-02-24.csv")
df1=pd.read_csv("/content/NiftyOptionCE_2025-02-25.csv")
df2=pd.read_csv("/content/NiftyOptionCE_2025-02-27.csv")

In [2]:
# prompt: ignaure warning

import warnings
warnings.filterwarnings('ignore')


In [3]:
CE_df=pd.concat([df,df1,df2])

In [4]:
CE_df.columns

Index(['Strike Price', 'expiryDate', 'openInterest', 'changeinOpenInterest',
       'pchangeinOpenInterest', 'totalTradedVolume', 'impliedVolatility',
       'lastPrice', 'change', 'pChange', 'totalBuyQuantity',
       'totalSellQuantity', 'bidQty', 'bidprice', 'askQty', 'askPrice',
       'underlyingValue', 'date', 'Time'],
      dtype='object')

In [5]:
# Standardize column names by removing spaces and making lowercase
CE_df.columns = CE_df.columns.str.replace(" ", "").str.lower()

# Convert 'date' and 'time' into a proper datetime column with corrected format
CE_df["datetime"] = pd.to_datetime(CE_df["date"] + " " + CE_df["time"], format="%Y-%m-%d %H-%M-%S")


In [6]:
CE_df.head()

,strikeprice,expirydate,openinterest,changeinopeninterest,pchangeinopeninterest,totaltradedvolume,impliedvolatility,lastprice,change,pchange,totalbuyquantity,totalsellquantity,bidqty,bidprice,askqty,askprice,underlyingvalue,date,time,datetime
0,17000,24-Dec-2025,9.333333,0.0,0.000000,0,0.00,0.00,0.00,0.000000,150,225,75,5802.65,75,6998.90,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45
1,18000,27-Mar-2025,6188.000000,79.0,1.293174,363,26.89,4701.15,-205.65,-4.191123,8550,9150,75,4699.60,75,4701.75,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45
2,18000,24-Dec-2025,125.000000,0.0,0.000000,0,0.00,0.00,0.00,0.000000,300,450,75,5350.00,75,5614.05,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45
3,18000,25-Jun-2026,1.333333,0.0,0.000000,0,0.00,0.00,0.00,0.000000,0,0,0,0.00,0,0.00,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45
4,19000,27-Mar-2025,818.333333,168.0,25.832906,205,0.00,3710.20,-209.80,-5.352041,9675,10575,75,3710.55,75,3721.20,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45


In [7]:
# prompt: convert expirydate column in to datetime column

import pandas as pd
# Convert 'expirydate' to datetime
CE_df["expirydate"] = pd.to_datetime(CE_df["expirydate"], format="%d-%b-%Y")
CE_df['date'] = pd.to_datetime(CE_df['date'], format='%Y-%m-%d')
CE_df['remaining']=CE_df['expirydate']-CE_df['date']
CE_df['remaining'] = CE_df['remaining'].dt.days

In [8]:
CE_df.head()

,strikeprice,expirydate,openinterest,changeinopeninterest,pchangeinopeninterest,totaltradedvolume,impliedvolatility,lastprice,change,pchange,...,totalsellquantity,bidqty,bidprice,askqty,askprice,underlyingvalue,date,time,datetime,remaining
0,17000,2025-12-24,9.333333,0.0,0.000000,0,0.00,0.00,0.00,0.000000,...,225,75,5802.65,75,6998.90,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,303
1,18000,2025-03-27,6188.000000,79.0,1.293174,363,26.89,4701.15,-205.65,-4.191123,...,9150,75,4699.60,75,4701.75,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,31
2,18000,2025-12-24,125.000000,0.0,0.000000,0,0.00,0.00,0.00,0.000000,...,450,75,5350.00,75,5614.05,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,303
3,18000,2026-06-25,1.333333,0.0,0.000000,0,0.00,0.00,0.00,0.000000,...,0,0,0.00,0,0.00,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,486
4,19000,2025-03-27,818.333333,168.0,25.832906,205,0.00,3710.20,-209.80,-5.352041,...,10575,75,3710.55,75,3721.20,22550.7,2025-02-24,15-18-45,2025-02-24 15:18:45,31


In [9]:
CE_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 484002 entries, 0 to 236585
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   strikeprice            484002 non-null  int64         
 1   expirydate             484002 non-null  datetime64[ns]
 2   openinterest           484002 non-null  float64       
 3   changeinopeninterest   484002 non-null  float64       
 4   pchangeinopeninterest  484002 non-null  float64       
 5   totaltradedvolume      484002 non-null  int64         
 6   impliedvolatility      484002 non-null  float64       
 7   lastprice              484002 non-null  float64       
 8   change                 484002 non-null  float64       
 9   pchange                484002 non-null  float64       
 10  totalbuyquantity       484002 non-null  int64         
 11  totalsellquantity      484002 non-null  int64         
 12  bidqty                 484002 non-null  int64    

In [10]:



# Select relevant columns with updated names
# df = df[["strikeprice", "expirydate", "datetime", "lastprice"]]

# Define function to resample into 5-minute candlestick format
def resample_candlestick(group):

    # group = group.set_index("datetime").sort_index()
    # return group.rolling("5min", closed="right").agg(
    #     {"lastprice": [lambda x: x.iloc[0], "max", "min", lambda x: x.iloc[-1]]}
    # ).dropna()

    resampled = group.resample("5min", closed="right", on="datetime").agg(
        {
            "lastprice": ["first", "max", "min", "last"],  # Open, High, Low, Close
            "underlyingvalue": ["first", "max", "min", "last"],
            "totaltradedvolume": "sum",  # Example: Summing volume
            "openinterest": "last",  # Example: Taking the last value of Open Interest
            "impliedvolatility":["first", "max", "min", "last"],
            'remaining': 'last'
        }
    ).dropna()
    resampled.index = resampled.index.shift(5, freq="min")
    return resampled

def rolling_stats(group):
    group = group.sort_values("datetime")
    for i in range(1, 11):  # Creating 10 lagged values
        group[f"VI_Open_{i}"] = group["VI_Open"].shift(i)
        group[f"VI_High_{i}"] = group["VI_High"].shift(i)
        group[f"VI_Low_{i}"] = group["VI_Low"].shift(i)
        group[f"VI_Close_{i}"] = group["VI_Close"].shift(i)
        group[f"Open_{i}"] = group["Open"].shift(i)
        group[f"High_{i}"] = group["High"].shift(i)
        group[f"Low_{i}"] = group["Low"].shift(i)
        group[f"Close_{i}"] = group["Close"].shift(i)
        group[f"underlyingValue_Open_{i}"] = group["underlyingValue_Open"].shift(i)
        group[f"underlyingValue_High_{i}"] = group["underlyingValue_High"].shift(i)
        group[f"underlyingValue_Low_{i}"] = group["underlyingValue_Low"].shift(i)
        group[f"underlyingValue_Close_{i}"] = group["underlyingValue_Close"].shift(i)
    return group



print(CE_df.columns)
# Apply groupby and resampling
candlestick_CE_df = CE_df.groupby(["strikeprice", "expirydate"]).apply(resample_candlestick)

# Reset index and rename columns
candlestick_CE_df.columns = ["Open", "High", "Low", "Close","underlyingValue_Open", "underlyingValue_High", "underlyingValue_Low", "underlyingValue_Close", "Total_Volume", "OI_Last","VI_Open", "VI_High", "VI_Low", "VI_Close",'remaining']
candlestick_CE_df.reset_index(inplace=True)

# prompt: add 4*10 column in df dataset which have 'VI_Open'	'VI_High'	'VI_Low'	'VI_Close' column of last 10 datapoint by groupby [strikeprice	expirydate] and sort by datetime

candlestick_CE_df = candlestick_CE_df.groupby(["strikeprice", "expirydate"]).apply(rolling_stats).reset_index(drop=True)

# prompt: create a column using time difference inbetween expirydate and datetime column in absolute minutes

# Calculate the time difference in minutes
candlestick_CE_df['time_diff_minutes'] = (candlestick_CE_df['expirydate'] - candlestick_CE_df['datetime']).dt.total_seconds() / 60

# Take the absolute value of the time difference
candlestick_CE_df['time_diff_minutes'] = candlestick_CE_df['time_diff_minutes'].abs()
candlestick_CE_df['type']=1
# Display the processed dataset
candlestick_CE_df.head()


Index(['strikeprice', 'expirydate', 'openinterest', 'changeinopeninterest',
       'pchangeinopeninterest', 'totaltradedvolume', 'impliedvolatility',
       'lastprice', 'change', 'pchange', 'totalbuyquantity',
       'totalsellquantity', 'bidqty', 'bidprice', 'askqty', 'askprice',
       'underlyingvalue', 'date', 'time', 'datetime', 'remaining'],
      dtype='object')


,strikeprice,expirydate,datetime,Open,High,Low,Close,underlyingValue_Open,underlyingValue_High,underlyingValue_Low,...,Open_10,High_10,Low_10,Close_10,underlyingValue_Open_10,underlyingValue_High_10,underlyingValue_Low_10,underlyingValue_Close_10,time_diff_minutes,type
0,17000,2025-06-26,2025-02-27 09:15:00,6850.0,6850.0,6850.0,6850.0,22547.55,22547.55,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170805.0,1
1,17000,2025-12-24,2025-02-24 15:20:00,0.0,0.0,0.0,0.0,22550.70,22551.20,22550.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435400.0,1
2,17000,2025-12-24,2025-02-24 15:25:00,0.0,0.0,0.0,0.0,22550.55,22550.55,22548.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435395.0,1
3,17000,2025-12-24,2025-02-24 15:30:00,0.0,0.0,0.0,0.0,22550.10,22550.10,22545.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435390.0,1
4,17000,2025-12-24,2025-02-24 15:35:00,0.0,0.0,0.0,0.0,22550.00,22550.00,22550.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435385.0,1


In [11]:
candlestick_CE_df[(candlestick_CE_df['strikeprice']==22700) & (candlestick_CE_df['expirydate']=='27-Feb-2025')]

,strikeprice,expirydate,datetime,Open,High,Low,Close,underlyingValue_Open,underlyingValue_High,underlyingValue_Low,...,Open_10,High_10,Low_10,Close_10,underlyingValue_Open_10,underlyingValue_High_10,underlyingValue_Low_10,underlyingValue_Close_10,time_diff_minutes,type
31319,22700,2025-02-27,2025-02-24 15:20:00,61.35,61.40,60.55,60.55,22550.70,22551.20,22550.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3400.0,1
31320,22700,2025-02-27,2025-02-24 15:25:00,61.05,61.05,58.95,59.55,22550.55,22551.20,22548.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3395.0,1
31321,22700,2025-02-27,2025-02-24 15:30:00,60.00,60.00,57.75,57.75,22548.90,22550.10,22545.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3390.0,1
31322,22700,2025-02-27,2025-02-24 15:35:00,57.30,57.30,56.60,56.60,22550.05,22550.05,22550.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3385.0,1
31323,22700,2025-02-27,2025-02-25 09:15:00,56.60,56.60,56.60,56.60,22553.35,22553.35,22553.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2325.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31452,22700,2025-02-27,2025-02-27 14:25:00,0.80,0.80,0.75,0.80,22542.20,22551.50,22542.20,...,1.20,1.30,1.20,1.30,22533.65,22534.35,22533.65,22534.35,865.0,1
31453,22700,2025-02-27,2025-02-27 14:30:00,0.75,0.80,0.50,0.55,22550.15,22550.15,22548.15,...,1.30,1.35,1.30,1.30,22540.30,22540.30,22538.70,22538.70,870.0,1
31454,22700,2025-02-27,2025-02-27 14:35:00,0.55,0.55,0.50,0.55,22546.15,22549.35,22546.15,...,1.10,1.30,1.05,1.10,22536.45,22536.55,22534.65,22536.55,875.0,1
31455,22700,2025-02-27,2025-02-27 14:40:00,0.45,0.45,0.40,0.45,22551.75,22551.75,22539.80,...,1.10,1.15,1.10,1.15,22536.05,22540.95,22536.05,22540.95,880.0,1


In [12]:
# # prompt: add 4*10 column in df dataset which have 'VI_Open'	'VI_High'	'VI_Low'	'VI_Close' column of last 10 datapoint by groupby [strikeprice	expirydate] and sort by datetime

# # Calculate rolling statistics for 'VI_Open', 'VI_High', 'VI_Low', 'VI_Close'
# def rolling_stats(group):
#     group = group.sort_values("datetime")
#     for i in range(1, 11):  # Creating 10 lagged values
#         group[f"VI_Open_{i}"] = group["VI_Open"].shift(i)
#         group[f"VI_High_{i}"] = group["VI_High"].shift(i)
#         group[f"VI_Low_{i}"] = group["VI_Low"].shift(i)
#         group[f"VI_Close_{i}"] = group["VI_Close"].shift(i)
#         group[f"Open_{i}"] = group["Open"].shift(i)
#         group[f"High_{i}"] = group["High"].shift(i)
#         group[f"Low_{i}"] = group["Low"].shift(i)
#         group[f"Close_{i}"] = group["Close"].shift(i)
#     return group

# candlestick_df = candlestick_df.groupby(["strikeprice", "expirydate"]).apply(rolling_stats).reset_index(drop=True)
# candlestick_df.head()


In [13]:
# a=candlestick_CE_df[(candlestick_CE_df['strikeprice']==22700) & (candlestick_CE_df['expirydate']=='27-Feb-2025')]
# a.to_csv('test.csv')

In [14]:
candlestick_CE_df.shape

(92573, 140)

In [15]:
df=pd.read_csv("/content/NiftyOptionPE_2025-02-24.csv")
df1=pd.read_csv("/content/NiftyOptionPE_2025-02-25.csv")
df2=pd.read_csv("/content/NiftyOptionPE_2025-02-27.csv")
PE_df=pd.concat([df,df1,df2])
# Standardize column names by removing spaces and making lowercase
PE_df.columns = PE_df.columns.str.replace(" ", "").str.lower()

# Convert 'date' and 'time' into a proper datetime column with corrected format
PE_df["datetime"] = pd.to_datetime(PE_df["date"] + " " + PE_df["time"], format="%Y-%m-%d %H-%M-%S")

# Convert 'expirydate' to datetime
PE_df["expirydate"] = pd.to_datetime(PE_df["expirydate"], format="%d-%b-%Y")
PE_df['date'] = pd.to_datetime(PE_df['date'], format='%Y-%m-%d')
PE_df['remaining']=PE_df['expirydate']-PE_df['date']
PE_df['remaining'] = PE_df['remaining'].dt.days

In [16]:


# Select relevant columns with updated names
# df = df[["strikeprice", "expirydate", "datetime", "lastprice"]]

# Define function to resample into 5-minute candlestick format
def resample_candlestick(group):

    # group = group.set_index("datetime").sort_index()
    # return group.rolling("5min", closed="right").agg(
    #     {"lastprice": [lambda x: x.iloc[0], "max", "min", lambda x: x.iloc[-1]]}
    # ).dropna()

    resampled = group.resample("5min", closed="right", on="datetime").agg(
        {
            "lastprice": ["first", "max", "min", "last"],  # Open, High, Low, Close
            "underlyingvalue": ["first", "max", "min", "last"],
            "totaltradedvolume": "sum",  # Example: Summing volume
            "openinterest": "last",  # Example: Taking the last value of Open Interest
            "impliedvolatility":["first", "max", "min", "last"],
            'remaining': 'last'
        }
    ).dropna()
    resampled.index = resampled.index.shift(5, freq="min")
    return resampled

def rolling_stats(group):
    group = group.sort_values("datetime")
    for i in range(1, 11):  # Creating 10 lagged values
        group[f"VI_Open_{i}"] = group["VI_Open"].shift(i)
        group[f"VI_High_{i}"] = group["VI_High"].shift(i)
        group[f"VI_Low_{i}"] = group["VI_Low"].shift(i)
        group[f"VI_Close_{i}"] = group["VI_Close"].shift(i)
        group[f"Open_{i}"] = group["Open"].shift(i)
        group[f"High_{i}"] = group["High"].shift(i)
        group[f"Low_{i}"] = group["Low"].shift(i)
        group[f"Close_{i}"] = group["Close"].shift(i)
        group[f"underlyingValue_Open_{i}"] = group["underlyingValue_Open"].shift(i)
        group[f"underlyingValue_High_{i}"] = group["underlyingValue_High"].shift(i)
        group[f"underlyingValue_Low_{i}"] = group["underlyingValue_Low"].shift(i)
        group[f"underlyingValue_Close_{i}"] = group["underlyingValue_Close"].shift(i)
    return group



print(PE_df.columns)
# Apply groupby and resampling
candlestick_PE_df = PE_df.groupby(["strikeprice", "expirydate"]).apply(resample_candlestick)

# Reset index and rename columns
candlestick_PE_df.columns = ["Open", "High", "Low", "Close","underlyingValue_Open", "underlyingValue_High", "underlyingValue_Low", "underlyingValue_Close", "Total_Volume", "OI_Last","VI_Open", "VI_High", "VI_Low", "VI_Close",'remaining']
candlestick_PE_df.reset_index(inplace=True)

# prompt: add 4*10 column in df dataset which have 'VI_Open'	'VI_High'	'VI_Low'	'VI_Close' column of last 10 datapoint by groupby [strikeprice	expirydate] and sort by datetime

candlestick_PE_df = candlestick_PE_df.groupby(["strikeprice", "expirydate"]).apply(rolling_stats).reset_index(drop=True)

# prompt: create a column using time difference inbetween expirydate and datetime column in absolute minutes

# Calculate the time difference in minutes
candlestick_PE_df['time_diff_minutes'] = (candlestick_PE_df['expirydate'] - candlestick_PE_df['datetime']).dt.total_seconds() / 60

# Take the absolute value of the time difference
candlestick_PE_df['time_diff_minutes'] = candlestick_PE_df['time_diff_minutes'].abs()
candlestick_PE_df['type']=-1
# Display the processed dataset
candlestick_PE_df.head()


Index(['strikeprice', 'expirydate', 'openinterest', 'changeinopeninterest',
       'pchangeinopeninterest', 'totaltradedvolume', 'impliedvolatility',
       'lastprice', 'change', 'pchange', 'totalbuyquantity',
       'totalsellquantity', 'bidqty', 'bidprice', 'askqty', 'askprice',
       'underlyingvalue', 'date', 'time', 'datetime', 'remaining'],
      dtype='object')


,strikeprice,expirydate,datetime,Open,High,Low,Close,underlyingValue_Open,underlyingValue_High,underlyingValue_Low,...,Open_10,High_10,Low_10,Close_10,underlyingValue_Open_10,underlyingValue_High_10,underlyingValue_Low_10,underlyingValue_Close_10,time_diff_minutes,type
0,12000,2026-06-25,2025-02-24 15:20:00,0.00,0.00,0.00,0.00,22550.70,22551.20,22550.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,698920.0,-1
1,12000,2026-06-25,2025-02-24 15:25:00,0.00,0.00,0.00,0.00,22550.55,22550.55,22548.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,698915.0,-1
2,12000,2026-06-25,2025-02-24 15:30:00,0.00,0.00,0.00,0.00,22550.10,22550.10,22545.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,698910.0,-1
3,12000,2026-06-25,2025-02-24 15:35:00,0.00,0.00,0.00,0.00,22550.00,22550.00,22550.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,698905.0,-1
4,12000,2026-06-25,2025-02-25 09:15:00,20.05,20.05,20.05,20.05,22553.35,22553.35,22553.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,697845.0,-1


In [17]:
candlestick=pd.concat([candlestick_CE_df,candlestick_PE_df])

In [18]:
a=candlestick[(candlestick['strikeprice']==22700) & (candlestick['expirydate']=='27-Feb-2025')]
a.to_csv('test.csv')

In [19]:
# prompt: find out duplicate column with same name

duplicate_columns = candlestick.columns[candlestick.columns.duplicated()]
duplicate_columns


Index([], dtype='object')

In [20]:
candlestick=candlestick.dropna().reset_index(drop=True)
# filtered_candlestick=candlestick[(candlestick['strikeprice']<=23500) & (candlestick['strikeprice']>=22000) ]

In [21]:
num_unique_values = candlestick[["strikeprice", "expirydate"]].drop_duplicates().shape[0]
print("Number of unique (strikeprice, expirydate) pairs:", num_unique_values)

Number of unique (strikeprice, expirydate) pairs: 826


In [22]:
# prompt: in a dataset, first sort and groupby  datetime column and then  return value one by one

import pandas as pd

def process_data(df):
    # Sort the DataFrame by the 'datetime' column
    df = df.sort_values('datetime')

    # Group the DataFrame by 'strikeprice' and 'expirydate'
    grouped = df.groupby(['datetime'])

    # Iterate through each group and return the values one by one
    for time,group_data in grouped:
        # for index, row in group_data.iterrows():

        underlying_value=group_data['underlyingValue_Close'].iloc[0]//50
        smaller_values = [underlying_value*50 - 50 * i for i in range(1, 11)]
        greater_values = [underlying_value*50 + 50 * i for i in range(1, 11)]
        strickPrice=greater_values+[underlying_value*50]+ smaller_values

        # print("Strike Prices:", strickPrice)

        sorted_datetime_list = sorted(group_data['expirydate'].unique())
        # print(sorted_datetime_list)
        # Reset index
        group_data.reset_index(drop=True, inplace=True)
        # print(group_data.columns)
        # Filter the data
        filtered_group_data = group_data[(group_data['strikeprice'].isin(strickPrice)) & (group_data['expirydate'].isin(sorted_datetime_list[:3]))]

        # Count unique values
        num_unique_values = filtered_group_data[["strikeprice", "expirydate"]].drop_duplicates().shape[0]
        # print("Number of unique (strikeprice, expirydate) pairs:", time, num_unique_values,
        #       filtered_group_data["strikeprice"].nunique(), filtered_group_data["expirydate"].nunique(),filtered_group_data.shape)

        # group_data=group_data.to_frame()
        filtered_group_data=filtered_group_data.sort_values(by=['strikeprice','remaining'])
        filtered_group_data=filtered_group_data.drop(columns=['expirydate','datetime'])
        yield filtered_group_data

# Example usage (assuming 'candlestick' is your DataFrame)
xx=process_data(candlestick)
i=0
# for xx in process_data(candlestick):
#     print(i)
#     i+=1

In [23]:
bb=next(xx)
bb

,strikeprice,Open,High,Low,Close,underlyingValue_Open,underlyingValue_High,underlyingValue_Low,underlyingValue_Close,Total_Volume,...,Open_10,High_10,Low_10,Close_10,underlyingValue_Open_10,underlyingValue_High_10,underlyingValue_Low_10,underlyingValue_Close_10,time_diff_minutes,type
145,22100,4.15,4.15,3.90,4.00,22583.45,22606.4,22583.45,22606.4,452269,...,7.10,7.15,6.95,6.95,22550.7,22551.2,22550.7,22550.7,2295.0,-1
809,22100,523.60,524.75,523.60,524.45,22583.45,22606.4,22583.45,22604.3,1086,...,521.20,521.20,521.20,521.20,22550.7,22551.2,22550.7,22550.7,2295.0,1
154,22100,32.80,32.80,31.85,31.90,22583.45,22606.4,22583.45,22606.4,17682,...,42.30,42.30,41.80,42.15,22550.7,22551.2,22550.7,22550.7,12375.0,-1
968,22100,602.00,602.00,602.00,602.00,22583.45,22606.4,22583.45,22606.4,320,...,605.00,605.00,605.00,605.00,22550.7,22551.2,22550.7,22550.7,12375.0,1
28,22100,68.70,70.00,68.50,68.50,22583.45,22606.4,22583.45,22606.4,2228,...,78.65,79.10,78.65,79.10,22550.7,22551.2,22550.7,22550.7,22455.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,23100,4.55,4.55,4.20,4.25,22583.45,22606.4,22583.45,22606.4,921375,...,7.15,7.20,6.90,6.95,22550.7,22551.2,22550.7,22550.7,2295.0,1
547,23100,35.05,35.05,34.15,34.60,22583.45,22606.4,22583.45,22606.4,43629,...,43.10,43.50,43.00,43.00,22550.7,22551.2,22550.7,22550.7,12375.0,1
550,23100,466.55,466.55,461.10,461.10,22583.45,22606.4,22583.45,22606.4,674,...,476.80,476.80,476.80,476.80,22550.7,22551.2,22550.7,22550.7,12375.0,-1
540,23100,83.85,85.65,83.80,85.65,22583.45,22606.4,22583.45,22606.4,2702,...,95.05,95.30,94.05,94.05,22550.7,22551.2,22550.7,22550.7,22455.0,1


In [24]:
bb

,strikeprice,Open,High,Low,Close,underlyingValue_Open,underlyingValue_High,underlyingValue_Low,underlyingValue_Close,Total_Volume,...,Open_10,High_10,Low_10,Close_10,underlyingValue_Open_10,underlyingValue_High_10,underlyingValue_Low_10,underlyingValue_Close_10,time_diff_minutes,type
145,22100,4.15,4.15,3.90,4.00,22583.45,22606.4,22583.45,22606.4,452269,...,7.10,7.15,6.95,6.95,22550.7,22551.2,22550.7,22550.7,2295.0,-1
809,22100,523.60,524.75,523.60,524.45,22583.45,22606.4,22583.45,22604.3,1086,...,521.20,521.20,521.20,521.20,22550.7,22551.2,22550.7,22550.7,2295.0,1
154,22100,32.80,32.80,31.85,31.90,22583.45,22606.4,22583.45,22606.4,17682,...,42.30,42.30,41.80,42.15,22550.7,22551.2,22550.7,22550.7,12375.0,-1
968,22100,602.00,602.00,602.00,602.00,22583.45,22606.4,22583.45,22606.4,320,...,605.00,605.00,605.00,605.00,22550.7,22551.2,22550.7,22550.7,12375.0,1
28,22100,68.70,70.00,68.50,68.50,22583.45,22606.4,22583.45,22606.4,2228,...,78.65,79.10,78.65,79.10,22550.7,22551.2,22550.7,22550.7,22455.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,23100,4.55,4.55,4.20,4.25,22583.45,22606.4,22583.45,22606.4,921375,...,7.15,7.20,6.90,6.95,22550.7,22551.2,22550.7,22550.7,2295.0,1
547,23100,35.05,35.05,34.15,34.60,22583.45,22606.4,22583.45,22606.4,43629,...,43.10,43.50,43.00,43.00,22550.7,22551.2,22550.7,22550.7,12375.0,1
550,23100,466.55,466.55,461.10,461.10,22583.45,22606.4,22583.45,22606.4,674,...,476.80,476.80,476.80,476.80,22550.7,22551.2,22550.7,22550.7,12375.0,-1
540,23100,83.85,85.65,83.80,85.65,22583.45,22606.4,22583.45,22606.4,2702,...,95.05,95.30,94.05,94.05,22550.7,22551.2,22550.7,22550.7,22455.0,1


In [25]:
bb.to_numpy().reshape(1,-1)

array([[ 2.21000e+04,  4.15000e+00,  4.15000e+00, ...,  2.25507e+04,
         2.24550e+04, -1.00000e+00]])

In [26]:
bb['strikeprice'].iloc[1]

22100

In [27]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is enabled
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of the GPU

True
1
Tesla T4


In [28]:
# import numpy as np
# import random
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from collections import deque

# # Define the DQN Model
# class DQN(nn.Module):
#     def __init__(self, state_size, action_size):
#         super(DQN, self).__init__()
#         self.fc1 = nn.Linear(state_size, 128)
#         self.fc2 = nn.Linear(128, 128)
#         self.fc3 = nn.Linear(128, action_size)

#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         return self.fc3(x)

# # Define Environment
# class OptionsTradingEnv:
#     def __init__(self, data=candlestick):
#         self.data=data

#         self.num_contracts = self.data[["strikeprice", "expirydate"]].drop_duplicates().shape[0]  # Number of contracts to track

#         self.state_size = 17388  # OHLC + contract details
#         self.action_size = 126 * 3  # (Neutral, Execute, Close)
#         self.time_step = 0
#         self.contracts = None   #self.generate_contracts()
#         self.open_positions = {}  # Track open contracts (id -> entry price)
#         self.nifty_data_at_5min=None


#     def generate_contracts(self):
#         """ Generate contract data with candlesticks """

#         # Sort the DataFrame by the 'datetime' column
#         df = self.data.sort_values('datetime')

#         # Group the DataFrame by 'strikeprice' and 'expirydate'
#         grouped = df.groupby(['datetime'])

#         # Iterate through each group and return the values one by one
#         for time,group_data in grouped:
#           # for index, row in group_data.iterrows():

#           underlying_value=group_data['underlyingValue_Close'].iloc[0]//50
#           smaller_values = [underlying_value*50 - 50 * i for i in range(1, 11)]
#           greater_values = [underlying_value*50 + 50 * i for i in range(1, 11)]
#           strickPrice=greater_values+[underlying_value*50]+ smaller_values

#           # print("Strike Prices:", strickPrice)

#           sorted_datetime_list = sorted(group_data['expirydate'].unique())
#           # print(sorted_datetime_list)
#           # Reset index
#           group_data.reset_index(drop=True, inplace=True)
#           # print(group_data.columns)

#           # Filter the data
#           filtered_group_data = group_data[(group_data['strikeprice'].isin(strickPrice)) & (group_data['expirydate'].isin(sorted_datetime_list[:3]))]

#           filtered_group_data=filtered_group_data.sort_values(by=['strikeprice','remaining'])
#           filtered_group_data=filtered_group_data.drop(columns=['expirydate','datetime'])
#           yield filtered_group_data

#     def step(self, action):
#         """ Apply action and update state """
#         self.time_step += 1
#         reward = 0
#         done=False



#         contract_id = action // 3  # Identify contract (0 to num_contracts - 1)
#         action_type = action % 3   # (0 = Neutral, 1 = Execute, 2 = Close)

#         if action_type == 1 and contract_id not in self.open_positions:  # Execute contract
#             self.open_positions[contract_id] = self.nifty_data_at_5min['Close'].iloc[contract_id]
#             reward = -self.nifty_data_at_5min['Close'].iloc[contract_id]  # Cost of buying the option

#         elif action_type == 2 and contract_id in self.open_positions:  # Close contract
#             exit_price = self.nifty_data_at_5min['Close'].iloc[contract_id]   # Closing price
#             entry_price = self.open_positions.pop(contract_id)
#             reward = (exit_price - entry_price) * (1 if self.nifty_data_at_5min['type'].iloc[contract_id] == 1 else -1)

#         # Remove expired contracts
#         # self.open_positions = {k: v for k, v in self.open_positions.items() if self.contracts[k]['time_diff_minutes'] > 885}
#         self.open_positions = {k: v for k, v in self.open_positions.items() if self.nifty_data_at_5min['remaining'].iloc[contract_id] >0}   # penalize to buy expiry contract

#         # Update state
#         try:
#           self.nifty_data_at_5min=next(self.contracts)
#         except StopIteration:
#           done=True

#         state = self.get_state()

#         return state, reward, done

#     def get_state(self):
#         """ Flatten contract attributes into a single state vector """

#         # repeated_cols = [col for col in self.nifty_data_at_5min.columns if 'underlyingValue' in col]
#         # first_values = self.nifty_data_at_5min[repeated_cols].iloc[0].values
#         # non_repeated_cols = [col for col in self.nifty_data_at_5min.columns if col not in repeated_cols]
#         # all_values = np.concatenate([first_values, self.nifty_data_at_5min[non_repeated_cols].values.flatten()])
#         vector_array = self.nifty_data_at_5min.to_numpy().reshape(1,-1)

#         return vector_array

#     def reset(self):
#         self.time_step = 0
#         self.open_positions = {}
#         self.contracts = self.generate_contracts()
#         self.nifty_data_at_5min=next(self.contracts)
#         return self.get_state()

# # Define DQN Agent
# class DQNAgent:
#     def __init__(self, state_size, action_size):
#         self.state_size = state_size
#         self.action_size = action_size
#         self.memory = deque(maxlen=2000)
#         self.gamma = 0.95  # Discount factor
#         self.epsilon = 1.0  # Exploration rate
#         self.epsilon_min = 0.01
#         self.epsilon_decay = 0.995
#         self.learning_rate = 0.001
#         self.model = DQN(state_size, action_size)
#         self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

#     def act(self, state):
#         """ Select action based on epsilon-greedy strategy """
#         if np.random.rand() <= self.epsilon:
#             return random.randrange(self.action_size)
#         state = torch.FloatTensor(state).unsqueeze(0)                           # converts the NumPy array or list state into a PyTorch tensor of type float32
#         with torch.no_grad():                                                   # It disables gradient computation in PyTorch. reducing memory usage and speeding up computations.
#             action_values = self.model(state[0])                                   #passing state as vector in nn and get output
#         return torch.argmax(action_values).item()                               #returns the index of the maximum value along a given dimension

#     def train(self, batch_size=32):
#         """ Train model using replay memory """
#         if len(self.memory) < batch_size:
#             return
#         batch = random.sample(self.memory, batch_size)
#         for state, action, reward, next_state in batch:
#             # not clear  ( calculate reward base on next state and update reward. but we don't have next state.)

#             target = reward + self.gamma * torch.max(self.model(torch.FloatTensor(next_state[0]).unsqueeze(0))).item()

#             # not clear
#             predicted = self.model(torch.FloatTensor(state[0]).unsqueeze(0))[0, action].float()

#             #calculate loss
#             loss = F.mse_loss(predicted, torch.tensor(target).float())

#             # update the weight after back propogation
#             self.optimizer.zero_grad()
#             loss.backward()
#             self.optimizer.step()

#     def remember(self, state, action, reward, next_state):
#         """ Store experience in memory """
#         self.memory.append((state, action, reward, next_state))

#     def update_epsilon(self):
#         """ Decay exploration rate """
#         self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

# # Train the agent
# env = OptionsTradingEnv(data=candlestick)
# print(env.state_size)
# agent = DQNAgent(env.state_size, env.action_size)

# episodes = 1000
# for episode in range(episodes):
#     state = env.reset()
#     total_reward = 0
#     while True:

#         action = agent.act(state)
#         # print(action)
#         next_state, reward, done = env.step(action)

#         agent.remember(state, action, reward, next_state)
#         state = next_state
#         total_reward += reward
#         # print(total_reward)
#         agent.train()
#         if done:
#             agent.update_epsilon()
#             break
#     print(f"Episode {episode + 1}, Total Reward: {total_reward:.2f}, Epsilon: {agent.epsilon:.3f}")


In [ ]:
%%time
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque

# Check and use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the DQN Model
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Define Environment
class OptionsTradingEnv:
    def __init__(self, data=candlestick):
        self.data=data

        self.num_contracts = self.data[["strikeprice", "expirydate"]].drop_duplicates().shape[0]  # Number of contracts to track

        self.state_size = 17388  # OHLC + contract details
        self.action_size = 126 * 3  # (Neutral, Execute, Close)
        self.time_step = 0
        self.contracts = None   #self.generate_contracts()
        self.open_positions = {}  # Track open contracts (id -> entry price)
        self.nifty_data_at_5min=None


    def generate_contracts(self):
        """ Generate contract data with candlesticks """

        # Sort the DataFrame by the 'datetime' column
        df = self.data.sort_values('datetime')

        # Group the DataFrame by 'strikeprice' and 'expirydate'
        grouped = df.groupby(['datetime'])

        # Iterate through each group and return the values one by one
        for time,group_data in grouped:
          # for index, row in group_data.iterrows():

          underlying_value=group_data['underlyingValue_Close'].iloc[0]//50
          smaller_values = [underlying_value*50 - 50 * i for i in range(1, 11)]
          greater_values = [underlying_value*50 + 50 * i for i in range(1, 11)]
          strickPrice=greater_values+[underlying_value*50]+ smaller_values

          # print("Strike Prices:", strickPrice)

          sorted_datetime_list = sorted(group_data['expirydate'].unique())
          # print(sorted_datetime_list)
          # Reset index
          group_data.reset_index(drop=True, inplace=True)
          # print(group_data.columns)

          # Filter the data
          filtered_group_data = group_data[(group_data['strikeprice'].isin(strickPrice)) & (group_data['expirydate'].isin(sorted_datetime_list[:3]))]

          filtered_group_data=filtered_group_data.sort_values(by=['strikeprice','remaining'])
          filtered_group_data=filtered_group_data.drop(columns=['expirydate','datetime'])
          yield filtered_group_data

    def step(self, action):
        """ Apply action and update state """
        self.time_step += 1
        reward = 0
        done=False



        contract_id = action // 3  # Identify contract (0 to num_contracts - 1)
        action_type = action % 3   # (0 = Neutral, 1 = Execute, 2 = Close)

        if action_type == 1 and contract_id not in self.open_positions:  # Execute contract
            self.open_positions[contract_id] = self.nifty_data_at_5min['Close'].iloc[contract_id]
            reward = -self.nifty_data_at_5min['Close'].iloc[contract_id]  # Cost of buying the option

        elif action_type == 2 and contract_id in self.open_positions:  # Close contract
            exit_price = self.nifty_data_at_5min['Close'].iloc[contract_id]   # Closing price
            entry_price = self.open_positions.pop(contract_id)
            reward = (exit_price - entry_price) * (1 if self.nifty_data_at_5min['type'].iloc[contract_id] == 1 else -1)

        # Remove expired contracts
        # self.open_positions = {k: v for k, v in self.open_positions.items() if self.contracts[k]['time_diff_minutes'] > 885}
        self.open_positions = {k: v for k, v in self.open_positions.items() if self.nifty_data_at_5min['remaining'].iloc[contract_id] >0}   # penalize to buy expiry contract

        # Update state
        try:
          self.nifty_data_at_5min=next(self.contracts)
        except StopIteration:
          done=True

        state = self.get_state()

        return state, reward, done

    def get_state(self):
        """ Flatten contract attributes into a single state vector """

        # repeated_cols = [col for col in self.nifty_data_at_5min.columns if 'underlyingValue' in col]
        # first_values = self.nifty_data_at_5min[repeated_cols].iloc[0].values
        # non_repeated_cols = [col for col in self.nifty_data_at_5min.columns if col not in repeated_cols]
        # all_values = np.concatenate([first_values, self.nifty_data_at_5min[non_repeated_cols].values.flatten()])
        vector_array = self.nifty_data_at_5min.to_numpy().reshape(1,-1)

        return vector_array

    def reset(self):
        self.time_step = 0
        self.open_positions = {}
        self.contracts = self.generate_contracts()
        self.nifty_data_at_5min=next(self.contracts)
        return self.get_state()



# Define DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001

        # Move model to GPU
        self.model = DQN(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def act(self, state):
        """ Select action based on epsilon-greedy strategy """
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        state = torch.FloatTensor(state).unsqueeze(0).to(device)  # Move to GPU
        with torch.no_grad():
            action_values = self.model(state)  # Get Q-values
        return torch.argmax(action_values).item()  # Return best action index

    def train(self, batch_size=32):
        """ Train model using replay memory """
        if len(self.memory) < batch_size:
            return
        batch = random.sample(self.memory, batch_size)

        # Convert batch data to tensors and move to GPU
        states, actions, rewards, next_states = zip(*batch)
        states = torch.FloatTensor(states).to(device)
        actions = torch.LongTensor(actions).to(device)  # Action index must be long type
        rewards = torch.FloatTensor(rewards).to(device)
        next_states = torch.FloatTensor(next_states).to(device)

        # Get Q-values for current states
        predicted_q_values = self.model(states)
        predicted_q_values = predicted_q_values.squeeze(1) # Now shape: (batch_size, action_size)


        predicted_q_values = predicted_q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        # Compute target Q-values
        with torch.no_grad():
            next_q_values = self.model(next_states)
            next_q_values = next_q_values.squeeze(1)

            max_next_q_values = torch.max(next_q_values, dim=1).values

            target_q_values = rewards + self.gamma * max_next_q_values

        # Compute loss
        loss = F.mse_loss(predicted_q_values, target_q_values)

        # Optimize model
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def remember(self, state, action, reward, next_state):
        """ Store experience in memory """
        self.memory.append((state, action, reward, next_state))

    def update_epsilon(self):
        """ Decay exploration rate """
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

# Initialize environment and agent
env = OptionsTradingEnv(data=candlestick)
agent = DQNAgent(env.state_size, env.action_size)

# Train the agent
episodes = 1000
for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    while True:
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        agent.remember(state, action, reward, next_state)
        state = next_state
        total_reward += reward
        agent.train()
        if done:
            agent.update_epsilon()
            break
    print(f"Episode {episode + 1}, Total Reward: {total_reward:.2f}, Epsilon: {agent.epsilon:.3f}")


Using device: cuda
Episode 1, Total Reward: -5889.60, Epsilon: 0.995
Episode 2, Total Reward: -6813.20, Epsilon: 0.990
Episode 3, Total Reward: -6111.50, Epsilon: 0.985
Episode 4, Total Reward: -4559.15, Epsilon: 0.980
Episode 5, Total Reward: -4442.95, Epsilon: 0.975
Episode 6, Total Reward: -7527.45, Epsilon: 0.970
Episode 7, Total Reward: -6534.65, Epsilon: 0.966
Episode 8, Total Reward: -7869.15, Epsilon: 0.961
Episode 9, Total Reward: -6175.95, Epsilon: 0.956
Episode 10, Total Reward: -8025.25, Epsilon: 0.951
Episode 11, Total Reward: -6823.95, Epsilon: 0.946
Episode 12, Total Reward: -7268.95, Epsilon: 0.942
Episode 13, Total Reward: -7228.95, Epsilon: 0.937
Episode 14, Total Reward: -8761.95, Epsilon: 0.932
Episode 15, Total Reward: -7131.80, Epsilon: 0.928
Episode 16, Total Reward: -4260.95, Epsilon: 0.923
Episode 17, Total Reward: -7755.90, Epsilon: 0.918
Episode 18, Total Reward: -7760.35, Epsilon: 0.914
Episode 19, Total Reward: -5462.25, Epsilon: 0.909
Episode 20, Total Rew

In [ ]:
# Save only the model parameters (recommended)
torch.save(agent.model.state_dict(), "dqn_model.pth")

# Save the entire model (optional, but not recommended for versioning)
torch.save(agent.model, "dqn_model_full.pth")
